# Notebook 09: Advanced Properties

## Beyond Basic Electronic Structure

**Prerequisites:** Completed stability tests (Notebook 07)

This notebook covers:
1. **Optical Properties** - Dielectric function, absorption spectra
2. **Phonon Properties** - Vibrational spectra, thermal properties
3. **Thermoelectric Properties** - Seebeck coefficient, transport

---

In [ ]:
import numpy as np
from typing import Dict, List, Tuple, Optional

# Physical constants
RY_TO_EV = 13.605693122994
BOHR_TO_ANGSTROM = 0.529177210903
KB_EV = 8.617333262e-5  # Boltzmann constant in eV/K
HBAR_EV_S = 6.582119569e-16  # Reduced Planck constant in eV·s

---

## 1. Optical Properties

### The Dielectric Function

The frequency-dependent dielectric function $\varepsilon(\omega)$ describes how a material responds to electromagnetic radiation:

$$\varepsilon(\omega) = \varepsilon_1(\omega) + i\varepsilon_2(\omega)$$

- $\varepsilon_1(\omega)$: Real part (dispersion, refraction)
- $\varepsilon_2(\omega)$: Imaginary part (absorption)

### From Band Structure to Optics

The imaginary part is calculated from interband transitions:

$$\varepsilon_2(\omega) \propto \sum_{c,v,\mathbf{k}} |\langle c\mathbf{k}|\mathbf{p}|v\mathbf{k}\rangle|^2 \delta(E_c - E_v - \hbar\omega)$$

where $c$ = conduction, $v$ = valence, $\mathbf{p}$ = momentum operator.

### Derived Optical Properties

| Property | Formula |
|----------|--------|
| Refractive index | $n = \sqrt{\frac{\varepsilon_1 + \sqrt{\varepsilon_1^2 + \varepsilon_2^2}}{2}}$ |
| Extinction coefficient | $k = \sqrt{\frac{-\varepsilon_1 + \sqrt{\varepsilon_1^2 + \varepsilon_2^2}}{2}}$ |
| Absorption coefficient | $\alpha = \frac{2\omega k}{c}$ |
| Reflectivity | $R = \frac{(n-1)^2 + k^2}{(n+1)^2 + k^2}$ |

In [ ]:
def generate_epsilon_input(prefix: str, outdir: str = './tmp',
                           calculation: str = 'eps', 
                           smearing: str = 'gaussian',
                           intersmear: float = 0.136,
                           wmin: float = 0.0, wmax: float = 30.0,
                           nw: int = 601) -> str:
    """
    Generate epsilon.x input for optical properties.
    
    Parameters
    ----------
    intersmear : float
        Interband smearing in eV (typical: 0.1-0.2 eV)
    wmin, wmax : float
        Energy range in eV
    nw : int
        Number of energy points
    
    Note: Requires NSCF calculation with many empty bands!
    """
    input_text = f"""&INPUTPP
    prefix = '{prefix}'
    outdir = '{outdir}'
    calculation = '{calculation}'
/
&ENERGY_GRID
    smeartype = '{smearing}'
    intersmear = {intersmear}d0
    wmin = {wmin}d0
    wmax = {wmax}d0
    nw = {nw}
/
"""
    return input_text

print("Optical Properties Input (epsilon.x)")
print("=" * 50)
print(generate_epsilon_input('silicon'))

In [ ]:
def calculate_optical_properties(eps1: np.ndarray, eps2: np.ndarray) -> Dict[str, np.ndarray]:
    """
    Calculate derived optical properties from dielectric function.
    
    Parameters
    ----------
    eps1 : np.ndarray
        Real part of dielectric function
    eps2 : np.ndarray
        Imaginary part of dielectric function
    
    Returns
    -------
    dict with n, k, alpha (relative), R
    """
    # Refractive index and extinction coefficient
    eps_mag = np.sqrt(eps1**2 + eps2**2)
    n = np.sqrt((eps1 + eps_mag) / 2)
    k = np.sqrt((-eps1 + eps_mag) / 2)
    
    # Reflectivity at normal incidence
    R = ((n - 1)**2 + k**2) / ((n + 1)**2 + k**2)
    
    return {
        'n': n,           # Refractive index
        'k': k,           # Extinction coefficient  
        'R': R,           # Reflectivity
        'eps_mag': eps_mag
    }

# Example: simplified model dielectric function
print("Optical Properties Calculation")
print("=" * 50)
print("\nExample: Model dielectric function")

# Simple Lorentz oscillator model (for demonstration)
omega = np.linspace(0.1, 10, 100)  # eV
omega0 = 3.4  # Gap energy (Si)
gamma = 0.1   # Broadening
wp = 15.0     # Plasma frequency parameter

# Model dielectric function
eps1_model = 1 + wp**2 * (omega0**2 - omega**2) / ((omega0**2 - omega**2)**2 + (gamma * omega)**2)
eps2_model = wp**2 * gamma * omega / ((omega0**2 - omega**2)**2 + (gamma * omega)**2)

props = calculate_optical_properties(eps1_model, eps2_model)
print(f"At ω = 1.5 eV (below gap):")
idx = np.argmin(np.abs(omega - 1.5))
print(f"  n = {props['n'][idx]:.3f}")
print(f"  k = {props['k'][idx]:.5f}")
print(f"  R = {props['R'][idx]:.3f}")

### Workflow for Optical Calculations

```
1. SCF calculation (standard)
   ↓
2. NSCF with MANY empty bands
   - nbnd = 3-4 × occupied bands
   - Dense k-mesh (12×12×12 or more)
   ↓
3. epsilon.x calculation
   - Computes ε₁(ω) and ε₂(ω)
   ↓
4. Post-process for optical properties
```

### Critical Parameters

| Parameter | Recommendation |
|-----------|---------------|
| K-points | Very dense (>10×10×10) |
| Empty bands | 3-4× occupied |
| Smearing | 0.1-0.2 eV (Gaussian) |
| Energy range | 0 to 2× gap energy |

---

## 2. Phonon Properties

### Why Phonons Matter

Phonons determine:
- **Thermal conductivity** - Heat transport
- **Specific heat** - Energy storage
- **Phase transitions** - Soft modes indicate instabilities
- **Superconductivity** - Electron-phonon coupling

### Phonon Dispersion

Phonon frequencies $\omega(\mathbf{q})$ are eigenvalues of the dynamical matrix:

$$D_{\alpha\beta}(\mathbf{q}) = \frac{1}{\sqrt{M_\alpha M_\beta}} \sum_R C_{\alpha\beta}(R) e^{i\mathbf{q}\cdot\mathbf{R}}$$

where $C_{\alpha\beta}$ are the force constants.

In [ ]:
def generate_ph_input(prefix: str, outdir: str = './tmp',
                      fildyn: str = 'dyn', ldisp: bool = True,
                      nq1: int = 4, nq2: int = 4, nq3: int = 4,
                      tr2_ph: float = 1e-14) -> str:
    """
    Generate ph.x input for phonon calculation.
    
    Parameters
    ----------
    ldisp : bool
        True for phonon dispersion on q-grid
    nq1, nq2, nq3 : int
        q-point grid for phonon calculations
    tr2_ph : float
        Self-consistency threshold for phonons
    """
    disp_params = ""
    if ldisp:
        disp_params = f"""    ldisp = .true.
    nq1 = {nq1}
    nq2 = {nq2}
    nq3 = {nq3}"""
    
    input_text = f"""Phonon calculation
&INPUTPH
    prefix = '{prefix}'
    outdir = '{outdir}'
    fildyn = '{fildyn}'
    tr2_ph = {tr2_ph:.1e}
{disp_params}
/
"""
    return input_text

def generate_q2r_input(fildyn: str = 'dyn', flfrc: str = 'fc') -> str:
    """
    Generate q2r.x input for real-space force constants.
    """
    return f"""&INPUT
    fildyn = '{fildyn}'
    flfrc = '{flfrc}'
/
"""

def generate_matdyn_input(flfrc: str = 'fc', flfrq: str = 'freq',
                          q_path: List[Tuple[str, float, float, float, int]] = None) -> str:
    """
    Generate matdyn.x input for phonon bands/DOS.
    
    Parameters
    ----------
    q_path : list of (label, qx, qy, qz, npoints)
        Path through BZ for phonon bands
    """
    input_text = f"""&INPUT
    flfrc = '{flfrc}'
    flfrq = '{flfrq}'
    asr = 'crystal'
/
"""
    
    if q_path:
        input_text += f"{len(q_path)}\n"
        for label, qx, qy, qz, npts in q_path:
            input_text += f"  {qx:.6f} {qy:.6f} {qz:.6f} {npts}  ! {label}\n"
    
    return input_text

print("Phonon Calculation Workflow")
print("=" * 50)
print("\n1. ph.x input:")
print(generate_ph_input('silicon', nq1=4, nq2=4, nq3=4))
print("\n2. q2r.x input:")
print(generate_q2r_input())
print("\n3. matdyn.x input (FCC path):")
fcc_qpath = [
    ('Gamma', 0.0, 0.0, 0.0, 20),
    ('X', 0.5, 0.5, 0.0, 20),
    ('W', 0.5, 0.75, 0.25, 20),
    ('L', 0.5, 0.5, 0.5, 20),
    ('Gamma', 0.0, 0.0, 0.0, 20),
    ('K', 0.375, 0.75, 0.375, 1),
]
print(generate_matdyn_input(q_path=fcc_qpath))

In [ ]:
def check_phonon_stability(frequencies: np.ndarray, threshold: float = -0.1) -> Tuple[bool, Dict]:
    """
    Check dynamic stability from phonon frequencies.
    
    Parameters
    ----------
    frequencies : np.ndarray
        Phonon frequencies in THz (or cm⁻¹)
    threshold : float
        Negative frequency threshold (allows for numerical noise)
    
    Returns
    -------
    is_stable : bool
    details : dict
    """
    min_freq = np.min(frequencies)
    imaginary_modes = frequencies[frequencies < threshold]
    
    is_stable = len(imaginary_modes) == 0
    
    details = {
        'min_frequency': min_freq,
        'n_imaginary': len(imaginary_modes),
        'imaginary_frequencies': imaginary_modes.tolist() if len(imaginary_modes) > 0 else [],
        'is_stable': is_stable
    }
    
    return is_stable, details

# Example
print("\nPhonon Stability Check")
print("=" * 50)

# Stable case
stable_freqs = np.array([0.0, 0.0, 0.0, 5.2, 5.2, 8.3, 12.1, 12.1, 15.4])
is_stable, details = check_phonon_stability(stable_freqs)
print(f"\nStable material: {is_stable}")
print(f"  Min frequency: {details['min_frequency']:.2f} THz")

# Unstable case (imaginary modes shown as negative)
unstable_freqs = np.array([-2.5, 0.0, 0.0, 4.1, 4.1, 7.2, 10.5, 10.5, 14.2])
is_stable, details = check_phonon_stability(unstable_freqs)
print(f"\nUnstable material: {is_stable}")
print(f"  Imaginary modes: {details['n_imaginary']}")
print(f"  Frequencies: {details['imaginary_frequencies']} THz")

### Thermal Properties from Phonons

Once phonon DOS $g(\omega)$ is known:

**Helmholtz free energy:**
$$F(T) = k_B T \int_0^{\infty} g(\omega) \ln\left[2\sinh\left(\frac{\hbar\omega}{2k_B T}\right)\right] d\omega$$

**Specific heat:**
$$C_V(T) = k_B \int_0^{\infty} g(\omega) \left(\frac{\hbar\omega}{k_B T}\right)^2 \frac{e^{\hbar\omega/k_B T}}{(e^{\hbar\omega/k_B T} - 1)^2} d\omega$$

In [ ]:
def calculate_specific_heat(omega: np.ndarray, dos: np.ndarray, 
                           temperatures: np.ndarray) -> np.ndarray:
    """
    Calculate specific heat from phonon DOS.
    
    Parameters
    ----------
    omega : np.ndarray
        Phonon frequencies in THz
    dos : np.ndarray
        Phonon density of states (states/THz)
    temperatures : np.ndarray
        Temperatures in K
    
    Returns
    -------
    Cv : np.ndarray
        Specific heat in units of kB
    """
    # Convert THz to eV: 1 THz = 4.136 meV
    thz_to_ev = 4.13567e-3
    omega_ev = omega * thz_to_ev
    
    Cv = np.zeros_like(temperatures)
    
    for i, T in enumerate(temperatures):
        if T < 1e-10:
            continue
        
        x = omega_ev / (KB_EV * T)  # ℏω / kT
        
        # Avoid overflow for large x
        valid = x < 50
        
        integrand = np.zeros_like(omega)
        integrand[valid] = (x[valid]**2 * np.exp(x[valid]) / 
                           (np.exp(x[valid]) - 1)**2 * dos[valid])
        
        Cv[i] = np.trapezoid(integrand, omega)
    
    return Cv

print("Thermal Properties from Phonons")
print("=" * 50)
print("\nSpecific heat: Cv(T) calculated from phonon DOS")
print("High-T limit: Cv → 3NkB (Dulong-Petit)")
print("Low-T limit: Cv ∝ T³ (Debye model)")

---

## 3. Thermoelectric Properties

### Boltzmann Transport Theory

Thermoelectric performance is characterized by the figure of merit:

$$ZT = \frac{S^2 \sigma T}{\kappa}$$

where:
- $S$ = Seebeck coefficient (thermopower)
- $\sigma$ = Electrical conductivity
- $\kappa$ = Thermal conductivity
- $T$ = Temperature

### Transport Coefficients

From Boltzmann transport (constant relaxation time τ approximation):

$$\sigma = e^2 \int \Xi(E) \left(-\frac{\partial f}{\partial E}\right) dE$$

$$S = -\frac{1}{eT\sigma} \int \Xi(E)(E-\mu) \left(-\frac{\partial f}{\partial E}\right) dE$$

where $\Xi(E)$ is the transport distribution function.

In [ ]:
def fermi_dirac(E: np.ndarray, mu: float, T: float) -> np.ndarray:
    """
    Fermi-Dirac distribution.
    
    Parameters
    ----------
    E : np.ndarray
        Energy in eV
    mu : float
        Chemical potential in eV
    T : float
        Temperature in K
    """
    x = (E - mu) / (KB_EV * T)
    # Prevent overflow
    x = np.clip(x, -500, 500)
    return 1 / (1 + np.exp(x))

def fermi_derivative(E: np.ndarray, mu: float, T: float) -> np.ndarray:
    """
    Negative derivative of Fermi-Dirac function (-df/dE).
    This is a peaked function around E = mu.
    """
    x = (E - mu) / (KB_EV * T)
    x = np.clip(x, -500, 500)
    f = 1 / (1 + np.exp(x))
    return f * (1 - f) / (KB_EV * T)

def estimate_seebeck_simple(dos: np.ndarray, energies: np.ndarray,
                           mu: float, T: float) -> float:
    """
    Estimate Seebeck coefficient from DOS (Mott formula approximation).
    
    S ≈ (π²/3) × (kB²T/e) × (d ln g(E)/dE)|_{E=μ}
    
    Parameters
    ----------
    dos : np.ndarray
        Density of states (states/eV)
    energies : np.ndarray
        Energy grid in eV
    mu : float
        Chemical potential in eV
    T : float
        Temperature in K
    
    Returns
    -------
    S : float
        Seebeck coefficient in μV/K
    """
    # Find DOS at Fermi level
    idx = np.argmin(np.abs(energies - mu))
    
    # Numerical derivative of ln(DOS) at mu
    dE = energies[1] - energies[0]
    
    # Avoid log of zero
    dos_safe = np.maximum(dos, 1e-10)
    
    if idx > 0 and idx < len(dos) - 1:
        d_ln_dos = (np.log(dos_safe[idx+1]) - np.log(dos_safe[idx-1])) / (2 * dE)
    else:
        d_ln_dos = 0
    
    # Mott formula: S = (π²/3) × (kB/e) × kB × T × d(ln g)/dE
    # kB in eV/K, result in V/K, convert to μV/K
    S = (np.pi**2 / 3) * KB_EV * T * d_ln_dos * 1e6
    
    return S

print("Thermoelectric Properties")
print("=" * 50)
print("\nSeebeck coefficient estimation from DOS")
print("Using Mott formula (valid for metals/degenerate semiconductors)")
print("\nFor accurate results: Use BoltzTraP2 or EPW codes")

### BoltzTraP2 Integration

For serious thermoelectric calculations, use BoltzTraP2:

```bash
# After QE calculation, generate BoltzTraP input
pw2wannier90.x < pw2wan.in > pw2wan.out
wannier90.x prefix
btp2 -vv interpolate prefix_band.dat
```

BoltzTraP2 provides:
- Seebeck coefficient $S(T, \mu)$
- Electrical conductivity $\sigma/\tau(T, \mu)$
- Electronic thermal conductivity $\kappa_e/\tau(T, \mu)$
- Power factor $S^2\sigma$

---

## 4. Complete Workflow for Advanced Properties

```
┌─────────────────────────────────────────────────────────────┐
│           ADVANCED PROPERTIES WORKFLOW                      │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  Prerequisites: Converged SCF + verified stability          │
│                                                             │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐         │
│  │  OPTICAL    │  │   PHONON    │  │ TRANSPORT   │         │
│  └──────┬──────┘  └──────┬──────┘  └──────┬──────┘         │
│         │                │                │                 │
│         ▼                ▼                ▼                 │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐         │
│  │ NSCF        │  │ ph.x       │  │ NSCF dense  │         │
│  │ Many bands  │  │ q-grid     │  │ k-mesh      │         │
│  └──────┬──────┘  └──────┬──────┘  └──────┬──────┘         │
│         │                │                │                 │
│         ▼                ▼                ▼                 │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐         │
│  │ epsilon.x   │  │ q2r.x      │  │ BoltzTraP2  │         │
│  │ ε(ω)        │  │ matdyn.x   │  │ S, σ, κ     │         │
│  └─────────────┘  └─────────────┘  └─────────────┘         │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

---

## Summary

### Key Points

1. **Optical properties** require NSCF with many empty bands
2. **Phonons** use DFPT (ph.x) - computationally expensive
3. **Thermoelectric** properties need dense k-meshes and specialized codes

### Computational Cost Comparison

| Property | Relative Cost | Key Requirement |
|----------|--------------|------------------|
| Band structure | 1× | - |
| DOS | 1× | Dense k-mesh |
| Optical | 3-5× | Many empty bands |
| Phonons (2×2×2) | 8× | DFPT at each q |
| Phonons (4×4×4) | 64× | - |
| Transport | 2-3× | Very dense k-mesh |

### Next Notebook
→ **10_Complete_Research_Workflow.ipynb**: Full workflow from structure to publication